In [3]:
import pandas as pd
import numpy as np
import requests
import searchtweets
from textblob import TextBlob
from afinn import Afinn
from datetime import *

In [4]:
df = pd.read_csv("~/work/projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv")
afinn = Afinn()
afinn_scores = [afinn.score(text) for text in df.text]
df['afinn'] = afinn_scores
df[['afinn', 'text']].head()

,afinn,text
0,0.0,Pourquoi ne donne t’on pas de l’Aspegic pédiat...
1,2.0,@mvrinesv Le syndrome grippal du Pfizer existe...
2,0.0,@montagnethiers #Orleat 210 personnes #vaccine...
3,-2.0,"En tout cas vendredi, c’est le vaccin astrazen..."
4,0.0,1700 centres Pfizer et Moderna...\nPlus de 70 ...


In [9]:
b = TextBlob("Je suis en colère")
b.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

# Création des métadonnées